In [11]:
import sys
import os
PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
print("Using project root:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])

Using project root: d:\Dev\ACM_auto_judge
sys.path[0]: d:\Dev\ACM_auto_judge


In [12]:
import pandas as pd
import joblib
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
df = pd.read_csv("../data/preprocessed_problems.csv")
print(df.shape)
print(df.columns)
df.head()

(4112, 10)
Index(['title', 'description', 'input_description', 'output_description',
       'sample_io', 'problem_class', 'problem_score', 'url', 'sample_io_text',
       'full_text'],
      dtype='object')


,title,description,input_description,output_description,sample_io,problem_class,problem_score,url,sample_io_text,full_text
0,uuu,unununium (uuu) was the name of the chemical e...,the input consists of one line with two intege...,the output consists of $m$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2\n2 3\n1 3\n3...",hard,9.7,https://open.kattis.com/problems/uuu,NaN,uuu unununium (uuu) was the name of the chemic...
1,house building,a number of eccentrics from central new york h...,"the input consists of $10$ test cases, which a...",print $k$ lines with the positions of the hous...,"[{'input': '0\n2 3 2\n50 60 50\n30 50 40', 'ou...",hard,9.7,https://open.kattis.com/problems/husbygge,NaN,house building a number of eccentrics from cen...
2,mario or luigi,mario and luigi are playing a game where they ...,NaN,NaN,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi,NaN,mario or luigi mario and luigi are playing a g...
3,the wire ghost,žofka is bending a copper wire. she starts wit...,the first line contains two integers $l$ and $...,the output consists of a single line consistin...,"[{'input': '4 3\n3 C\n2 C\n1 C', 'output': 'GH...",hard,9.6,https://open.kattis.com/problems/thewireghost,NaN,the wire ghost žofka is bending a copper wire....
4,barking up the wrong tree,"your dog spot is let loose in the park. well, ...",the first line of input consists of two intege...,write a single line containing the length need...,"[{'input': '2 0\n10 0\n10 10', 'output': '14.1...",hard,9.6,https://open.kattis.com/problems/barktree,NaN,barking up the wrong tree your dog spot is let...


In [13]:
from utils.feature_engineering import (
    text_length,
    word_count,
    math_symbol_count,
    keyword_features,
    constraint_features,
    input_structure_features,
    algorithmic_depth,
    DIFFICULTY_KEYWORDS,
    CONSTRAINT_FEATURE_NAMES,
    STRUCTURE_FEATURE_NAMES
)

In [14]:
df = pd.read_csv("../data/preprocessed_problems.csv")

In [15]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=3,
    stop_words="english"
)
X_tfidf = tfidf.fit_transform(df["full_text"])

In [16]:
df["text_length"] = df["full_text"].apply(text_length)
df["word_count"] = df["full_text"].apply(word_count)
df["math_symbol_count"] = df["full_text"].apply(math_symbol_count)
df["algorithmic_depth"] = df["full_text"].apply(algorithmic_depth)
keyword_df = pd.DataFrame(
    df["full_text"].apply(keyword_features).tolist(),
    columns=[f"kw_{k.replace(' ', '_')}" for k in DIFFICULTY_KEYWORDS]
)
constraint_df = pd.DataFrame(
    df["full_text"].apply(constraint_features).tolist(),
    columns=CONSTRAINT_FEATURE_NAMES
)
structure_df = pd.DataFrame(
    df["full_text"].apply(input_structure_features).tolist(),
    columns=STRUCTURE_FEATURE_NAMES
)
dense_df = pd.concat(
    [
        df[["text_length", "word_count", "math_symbol_count", "algorithmic_depth"]],
        keyword_df,
        constraint_df,
        structure_df
    ],
    axis=1
)
dense_df["text_length"] *= 0.1
dense_df["word_count"] *= 0.1
X_dense = csr_matrix(dense_df.values)
X_full = hstack([X_tfidf, X_dense])
joblib.dump(tfidf, "../models/tfidf_vectorizer.pkl")
# joblib.dump(X_tfidf, "../models/X_tfidf.pkl")
# joblib.dump(X_dense, "../models/X_dense.pkl")
joblib.dump(X_full, "../models/X_full.pkl")
joblib.dump(y_class, "../models/y_class.pkl")
joblib.dump(y_score, "../models/y_score.pkl")
print("✅ Features saved")
print("Final X shape:", X_full.shape)

✅ Features saved
Final X shape: (4112, 5027)


In [17]:
y_class = df["problem_class"]
y_score = df["problem_score"]